In [34]:
import pandas as pd
import re
import numpy as np
import pickle
import nltk
import spacy
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from catboost import CatBoostRegressor
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import STOPWORDS
%matplotlib inline
#nltk.download('punkt')
import tensorflow_hub as hub
import tensorflow as tf


In [35]:
### genero un nuevo feature que es saber si la location es nula o no, luego tomo un embeding de texto y lo combino con un xgb

In [36]:
train = pd.read_csv('csv/train.csv')
test  = pd.read_csv('csv/test.csv')

In [ ]:
train.sample(n = 5)

In [ ]:
train['location'].isnull().mean() # tenemos un 30% de locations que son nul

In [ ]:
train['loc_nula'] = train.apply(lambda x: ( 1 if pd.isna(x['location']) else 0  ), axis = 1 )

In [ ]:
train.sample(10)

In [ ]:
#hago lo mismo para test.

In [ ]:
test['loc_nula'] = test.apply(lambda x: ( 1 if pd.isna(x['location']) else 0  ), axis = 1 )

In [ ]:
#### importo el embeding de w2v
w2v = pd.read_csv('csv/solo_embedding_word2vec_train.csv')
w2v_test = pd.read_csv('csv/solo_embedding_word2vec_test.csv')

In [ ]:
w2v.sample()

In [ ]:
train['w2v'] = w2v['word2vec_score']

In [ ]:
train.head()

In [ ]:
train_limpio = train[ ['loc_nula', 'w2v']]

In [ ]:
train_limpio.head()

In [ ]:
#Aplico lo mismo a test.
test['w2v'] = w2v_test['word2vec_score']

In [ ]:
test_limpio = test[ ['loc_nula', 'w2v']]

In [ ]:
X, y = train_limpio, train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 5)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
rf_model = RandomForestRegressor(random_state=13, n_estimators=5, max_depth=10)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
lgb_class = lgb.LGBMRegressor(learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 5)
lgb_class.fit(X_train, y_train)
preds = lgb_class.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
catb = CatBoostRegressor(iterations=5, depth=10)
catb.fit(X_train, y_train)
preds = catb.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
gb = GradientBoostingRegressor(n_estimators=5, learning_rate=0.1, 
                                max_features=2, max_depth = 10, random_state = 0)
gb.fit(X_train, y_train)
preds = gb.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
eclf2 = VotingRegressor(estimators=[
         ('xgb', xg_reg), ('rf', rf_model), ('catb', catb), ('gb', gb), ('lgbm',lgb_class)])

eclf2 = eclf2.fit(X, y)
preds = eclf2.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

### Predicciones, me quedo con Voting regresor que fue el mejor puntaje

In [ ]:
test_limpio.head() #esta correctamente formateado.

In [ ]:
test['target'] = eclf2.predict(test_limpio)

In [ ]:
test.head()

In [ ]:
test = test [ ['id','target'] ]

In [ ]:
test ['target']   = test['target'].round().astype(int)

In [ ]:
test.head()

In [ ]:
test.set_index('id',inplace= True)
test.head()

In [ ]:
test.to_csv('csv/summit_geo_binario_w2v_nico.csv')